In [1]:
# Import relevant packages
import tensorflow as tf
import numpy as np
import h5py
import scipy.io
import skimage.io 

In [2]:
# Load data and set up sets
tf.reset_default_graph()
f = h5py.File('data_all.h5', 'r')

X_train = f['train_images'].value
y_train = f['train_labels'].value

X_val = f['validation_images'].value
y_val = f['validation_labels'].value

/usr/local/lib/python3.5/dist-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [10]:
# Define a U-net model
def make_unet():
    inputs = tf.keras.layers.Input(shape=(512, 192, 3))
    
    conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(pool2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same')(pool3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same')(conv4)
    drop4 = tf.keras.layers.Dropout(0.5)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same')(pool4)
    conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same')(conv5)
    drop5 = tf.keras.layers.Dropout(0.5)(conv5)

    up6 = tf.keras.layers.Conv2D(512, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(drop5))
    merge6 = tf.keras.layers.concatenate([conv4,up6], axis = 3)
    conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same')(merge6)
    conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same')(conv6)

    up7 = tf.keras.layers.Conv2D(256, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv6))
    merge7 = tf.keras.layers.concatenate([conv3,up7], axis = 3)
    conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(merge7)
    conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same')(conv7)

    up8 = tf.keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv7))
    merge8 = tf.keras.layers.concatenate([conv2,up8], axis = 3)
    conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(merge8)
    conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(conv8)

    up9 = tf.keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv8))
    merge9 = tf.keras.layers.concatenate([conv1,up9], axis = 3)
    conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(merge9)
    conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')(conv9)
    conv9 = tf.keras.layers.Conv2D(2, 3, activation = 'relu', padding = 'same')(conv9)
    conv10 = tf.keras.layers.Conv2D(1, 1, activation = 'sigmoid')(conv9)
    conv10 = tf.keras.layers.Reshape((conv10.shape[1], conv10.shape[2]))(conv10)
    model = tf.keras.Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = tf.train.AdamOptimizer(0.00001), loss = 'mean_squared_error')
    
    model.summary()

    return model

model = make_unet()
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=5)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs/{}'.format('model_name'))
checkpoint = tf.keras.callbacks.ModelCheckpoint('./model.{epoch:02d}-{val_loss:.10f}.hdf5', monitor='val_loss', verbose=0, period=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 512, 192, 3)  0                                            
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 512, 192, 64) 1792        input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_73 (Conv2D)              (None, 512, 192, 64) 36928       conv2d_72[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 256, 96, 64)  0           conv2d_73[0][0]                  
__________________________________________________________________________________________________
conv2d_74 

In [11]:
hist = model.fit(X_train, y_train,
          batch_size=10,
          epochs=100,
          verbose=1,
          validation_data=(X_val, y_val),
          callbacks=[early_stop, tensorboard, checkpoint])

Train on 490 samples, validate on 10 samples
Epoch 1/100
490/490 [==============================] - 131s 268ms/sample - loss: 0.2453 - val_loss: 0.1797
Epoch 2/100
490/490 [==============================] - 112s 229ms/sample - loss: 0.0208 - val_loss: 0.0072
Epoch 3/100
490/490 [==============================] - 111s 227ms/sample - loss: 0.0029 - val_loss: 0.0071
Epoch 4/100
490/490 [==============================] - 111s 227ms/sample - loss: 0.0029 - val_loss: 0.0071
Epoch 5/100
490/490 [==============================] - 111s 226ms/sample - loss: 0.0029 - val_loss: 0.0070
Epoch 6/100
490/490 [==============================] - 111s 226ms/sample - loss: 0.0029 - val_loss: 0.0070
Epoch 7/100
490/490 [==============================] - 110s 225ms/sample - loss: 0.0029 - val_loss: 0.0070
Epoch 8/100
490/490 [==============================] - 110s 225ms/sample - loss: 0.0029 - val_loss: 0.0070
Epoch 9/100
490/490 [==============================] - 110s 224ms/sample - loss: 0.0029 - val_loss:

In [12]:
tf.keras.models.save_model(model,'full_model.h5')

In [17]:
# Make predictions
X_pred = model.predict(X_train)
X_pred_too = model.predict(X_val)
X_pred_all = np.concatenate((X_pred,X_pred_too),axis=0)

In [28]:
# Write data
hf = h5py.File('model_results.h5', 'w')
hf.create_dataset('predictions', data=X_pred_all)
hf.close()